In [42]:
import pandas as pd

# data/Amazon Import - 01.xlsx - import .xlsx file 


In [52]:
# import chardet

# def find_encoding(fname):
#     r_file = open(fname, 'rb').read()
#     result = chardet.detect(r_file)
#     return result['encoding']

# encoding = find_encoding('data/All_Listings_Report_MX.txt')
# print(encoding)

UTF-8-SIG


In [44]:

import_df_MT = pd.read_excel('data/Amazon Import - 01.xlsx', sheet_name='Master Sku Translation')

# save import_df_MT to csv file in results folder
import_df_MT.to_csv('results/import_df_MT.csv', index=False)

all_listings_US  = pd.read_csv('data/All_Listings_Report_US.txt', sep='\t', encoding='Windows-1252')
all_listings_CA = pd.read_csv('data/All_Listings_Report_CA.txt', sep='\t', encoding='UTF-8-SIG')

#data/STOCK-STATUS2024030611.899373.TXT

BS_export_df = pd.read_csv('data/STOCK-STATUS202407098.952568.TXT', sep='\t', encoding='ascii', skiprows=2)

# save BS_export_df to csv file in results folder
BS_export_df.to_csv('results/BS_export_df.csv', index=False)

# print(import_df_MT.columns)

# print(all_listings_US.columns)

# print(all_listings_CA.columns)


# print(BS_export_df.columns)

BS_sku_list = BS_export_df['CODE'].str.strip().tolist()

print(BS_sku_list)

['FH37516S', '.DEFECT', '.DELIVERY', '.DISCOUNT', '.PRICEERROR', '.RETURNS', '001BF', '001CY', '001GF', '001OB', '001PB', '002BS', '009CB', '018OG', '02579-05', '02579-14', '02579-14S', '02579-18', '02579-18S', '02579-20S', '02579-40', '02579-40S', '05-131', '05-216', '05-236', '05-321', '05-552', '050.006', '051.019', '051.038', '051.041', '051.042', '051.045', '077GM', '07CLEARANCE', '090313', '090344', '094362', '095271', '095318', '095363', '10004-01', '10004-02', '10004-02S', '10004-05', '10004-07', '10004-08', '10004-10', '10004-14', '10004-15', '10004-15S', '10004-18', '10004-19', '10004-25', '10004-26', '10004-30', '10004-31', '10004-37', '10004-37S', '10004-38', '10004-39', '10004-44', '10004-47', '10006-11', '10006-14', '10006-15', '10006-15S', '10006-18', '10006-31', '10028-01', '10028-02', '10028-10', '10028-10S', '10028-14', '10028-15', '10028-19', '10028-19S', '10028-29', '10028-30', '10028-31', '10028-38', '10028-45', '10045-10', '10045-10S', '10045-15', '10045-15S', '10

In [45]:
import_df_MT = import_df_MT[['seller-sku', 'SKU (FINAL)','Product','Product NZ','Type']] 

#rename columns to amazon_sku, bs_sku, p_sku, nz_sku, type
import_df_MT.columns = ['amazon_sku', 'bs_sku', 'p_sku', 'nz_sku', 'type']

In [46]:


# save to amazon_sku_mapping.csv file in results folder
import_df_MT.to_csv('results/amazon_sku_mapping.csv', index=False)


In [47]:
all_listings_US = all_listings_US[['asin1', 'seller-sku','status']]
all_listings_CA = all_listings_CA[['asin1', 'seller-sku','status']]
all_listings_US.columns = ['ASIN', 'amazon_sku','status']
all_listings_CA.columns = ['ASIN', 'amazon_sku','status']

In [48]:
# compare ASINs in all_listings_US and all_listings_CA find nomber of matching ASINs and not matching ASINs
matching_asins = pd.merge(all_listings_US, all_listings_CA, on='ASIN', how='inner')
not_matching_asins = pd.concat([all_listings_US, all_listings_CA]).drop_duplicates(keep=False)

# print number of matching and not matching ASINs
print(matching_asins.shape[0])
print(not_matching_asins.shape[0])

38686
11822


In [49]:

def merge_dataframes(df1, df2):
    # Merge the two DataFrames on the 'ASIN' column
    merged_df = pd.merge(df1, df2, on='ASIN', how='outer', suffixes=('_df1', '_df2'))

    not_match_bs_sku = []

    # List of columns to check
    columns_to_check = ['bs_sku', 'p_sku', 'nz_sku', 'type']

    for column in columns_to_check:
        # Create new column names for the merged DataFrame
        column_df1 = f'{column}_df1'
        column_df2 = f'{column}_df2'

        # Fill the column with values from the two original DataFrames
        for index, row in merged_df.iterrows():
            if pd.isna(row[column_df1]) and pd.notna(row[column_df2]):
                merged_df.at[index, column] = row[column_df2]
            elif pd.notna(row[column_df1]) and pd.isna(row[column_df2]):
                merged_df.at[index, column] = row[column_df1]
            elif pd.notna(row[column_df1]) and pd.notna(row[column_df2]) and row[column_df1] == row[column_df2]:
                merged_df.at[index, column] = row[column_df1]
            elif pd.isna(row[column_df1]) and pd.isna(row[column_df2]):
                merged_df.at[index, column] = ''
            else:
                print(f"ASIN: {row['ASIN']} , column: {column} - '{row[column_df1]}' != '{row[column_df2]}'")
                if column == 'bs_sku':
                    not_match_bs_sku.append([row['ASIN'], row[column_df1], row[column_df2]])
                    
                    


        # Drop the temporary columns
        merged_df = merged_df.drop(columns=[column_df1, column_df2])

    # Save the mismatched rows to a CSV file
    if not_match_bs_sku:
        not_match_bs_sku_df = pd.DataFrame(not_match_bs_sku, columns=['ASIN', 'bs_sku_df1', 'bs_sku_df2'])
        not_match_bs_sku_df.to_csv('results/not_match_bs_sku.csv', index=False)

    return merged_df
 



In [50]:
#generate ASIN to sku mapping file for US an CA from import_df_MT


all_listings_US = all_listings_US.merge(import_df_MT, on='amazon_sku', how='left')
all_listings_CA = all_listings_CA.merge(import_df_MT, on='amazon_sku', how='left')


# bs sku to list and remove nan values
list_of_bs_skus = all_listings_US['bs_sku'].str.strip().tolist()
list_of_bs_skus = [x for x in list_of_bs_skus if str(x) != 'nan']

print(list_of_bs_skus)

['001BF', '001BF', '001BFS', '001CY', '001GF', '001GF', '001GF', '001GFS', '001OB', '001OB', '001OBS', '001OBS', '001PB', '001PB', '001PB', '001PBS', '002BS', '002BS', '002BS', '002BS', '002BS', '003BS', '009CB', '009CBS', '018OG', '018OG', '018OG', '018OGS', '018OGS', '077GM', '077GMS', '149TO', '149TO', '149TO', '149TOS', '186FB', '186FB', '186FBS', '237SR', '237SR', '237SRS', '237SRS', '241PG', '241PG', '241PGS', '242PP', '242PP', '242PPS', '243FG', '243FG', '243FGS', '244WGS', '245WB', '245WB', '245WBS', '248GR', '248GRS', '248GRS', '313TTS', '315TF', '315TF', '315TF', '315TF', '315TFS', '315TFS', '37855-1S', '37855-2S', '37855-3S', '37855-4S', '37855-5S', '37855-6S', '37857-1S', '37857-2S', '37857-3', '37857-3S', '37857-4S', '37857-5S', '37857-6S', '37858-1S', '37858-2S', '37858-3S', '37859-1S', '37859-3S', '37859-4S', '37859-5S', '37859-6S', '37860-1S', '37860-2S', '37860-4S', '37861-1S', '37861-2S', '37861-3S', '37861-4S', '37862-1S', '37862-2S', '37862-3S', '37862-4S', '37863-1

In [51]:


# find num sku in list_of_bs_skus not in BS_sku_list
not_in_BS_sku_list = [x for x in list_of_bs_skus if x not in BS_sku_list]
print(not_in_BS_sku_list)


# drop amazon_sku column
# all_listings_US = all_listings_US.drop(columns='amazon_sku')
# all_listings_CA = all_listings_CA.drop(columns='amazon_sku')

# Merge the 'all_listings_US' and 'all_listings_CA' dataframes based on the 'ASIN' column. Follow these rules:
# 1. If an 'ASIN' value matches in both dataframes:
#    a. If 'bs_sku', 'p_sku', 'nz_sku', 'type' columns have the same values in both dataframes, retain the value.
#    b. If 'bs_sku', 'p_sku', 'nz_sku', 'type' columns have different values in both dataframes, print the 'ASIN' and column name to the console.
#    c. If a column value is missing in one dataframe, use the value from the other dataframe.
#    d. If a column value is missing in both dataframes, keep the value as empty.
# 2. If an 'ASIN' value is only present in one dataframe, retain all the values from that dataframe.



merged_df = merge_dataframes(all_listings_US, all_listings_CA)

['001BFS', '001GFS', '001OBS', '001OBS', '001PBS', '003BS', '009CBS', '018OGS', '018OGS', '077GMS', '149TOS', '186FBS', '237SRS', '237SRS', '241PGS', '242PPS', '243FGS', '244WGS', '245WBS', '248GRS', '248GRS', '313TTS', '315TFS', '315TFS', '37855-1S', '37855-2S', '37855-3S', '37855-4S', '37855-6S', '37857-1S', '37857-6S', '37858-1S', '37858-2S', '37859-1S', '37859-5S', '37860-1S', '37860-2S', '37861-4S', '37862-2S', '37863-2S', '37863-4S', '37864-1S', '37864-2S', '37864-4S', '37865-2S', '37865-3S', '37865-4S', '37865-6S', '37865-7S', '37866-1S', '37866-2S', '37866-3S', '37866-4S', '37866-5S', '37867-1S', '37867-3S', '37868-1S', '37868-2S', '37869-2S', '37901-1S', '37901-4S', '37901-5S', '37902-1S', '37902-2S', '37902-3S', '37902-4S', '37903-1S', '37903-2S', '37903-3S', '37903-4S', '37903-5S', '37904-1S', '37904-2S', '37904-3S', '37904-4S', '37904-5S', '37904-6S', '37904-7S', '37904-8S', '37912-2S', '37913-2S', '37913-5S', '37919-2S', '37953-6S', '37953-7S', '37954-3S', 'SM578B', '537SM